## Partie 1 : Préparation des données communales

In [83]:
import pandas as pd
import geopandas as gpd
import numpy as np
import unicodedata,time,os,csv
from datetime import datetime
import requests
import os, glob
import io
from zipfile import ZipFile
from pathlib import Path

In [84]:
os.chdir('/home/manon/Documents/Scripts/1_Prepare_urban_environments/')

Chargement données des aires d'attraction des villes (INSEE, 2020)

In [85]:
url =  r"https://www.insee.fr/fr/statistiques/fichier/4803954/AAV2020_au_01-01-2022.zip"
r = requests.get(url, allow_redirects=True)

def read_zip(zip_fn, extract_fn=None):
    zf = ZipFile(zip_fn)
    if extract_fn:
        return zf.read(extract_fn)
    else:
        return {name:zf.read(name) for name in zf.namelist()}
    
open('Data/AAV2020_au_01-01-2022.zip', 'wb').write(r.content)
df_ZAU = pd.read_excel(io.BytesIO(read_zip(r'Data/AAV2020_au_01-01-2022.zip', 'AAV2020_au_01-01-2022.xlsx')),sheet_name=1,skiprows=5)

Chargement de la couche des communes en shapefile

In [86]:
url =  r"https://www.insee.fr/fr/statistiques/fichier/4803954/fonds_aav2020_2022.zip"
r = requests.get(url, allow_redirects=True)

open('Data/fonds_aav2020_2022.zip', 'wb').write(r.content)
ZipFile('Data/fonds_aav2020_2022.zip').extractall(path="./Data/")
ZipFile('Data/com_aav2020_2022.zip').extractall(path="./Data/")

df_com_shp = gpd.GeoDataFrame.from_file("./Data/com_aav2020_2022.shp")
df_com_shp.rename(columns={'codgeo':'CODGEO'},inplace=True)

Chargement des données de population (recensement INSEE, 2019)

In [87]:
url = r'https://www.insee.fr/fr/statistiques/fichier/6011070/ensemble.zip'
r = requests.get(url, allow_redirects=True)

open('Data/ensemble.zip', 'wb').write(r.content)
df_com_pop = pd.read_csv(io.BytesIO(read_zip(r'Data/ensemble.zip', 'donnees_communes.csv')), delimiter=';')

Préparation des données de population : création colonne CODGEO (code INSEE) et suppression des arrondissements (pour que les données soient compatibles avec le fichier des aires d'attraction et celui des géométries)

In [88]:
# Code INSEE
df_com_pop['CODDEP']=df_com_pop['CODDEP'].apply(str)
df_com_pop['LEN_CODDEP'] = df_com_pop['CODDEP'].apply(len) 

df_com_pop['CODGEO'] = np.where(df_com_pop['LEN_CODDEP']<3, df_com_pop['CODDEP'] + df_com_pop['CODCOM'].apply(str).str.zfill(3), 
                            df_com_pop['CODDEP'].apply(str).str[:2] + df_com_pop['CODCOM'].apply(str).str.zfill(3))

df_com_pop.drop(columns=['CODARR', 'CODCAN', 'CODCOM', 'PMUN', 'PCAP', 'LEN_CODDEP'],inplace= True)

In [89]:
# Gestion des arrondissements : calcul population Paris, Lyon, Marseille + suppression des arrondissements
df_com_arr = df_com_pop[df_com_pop['COM'].str.contains('Arrondissement')]
df_com_arr['COM']=np.where(df_com_arr['CODDEP']=='75', 'Paris',
              np.where(df_com_arr['CODDEP']=='69', 'Lyon','Marseille'))
df_com_arr['CODGEO']=np.where(df_com_arr['CODDEP']=='75', '75056',
              np.where(df_com_arr['CODDEP']=='69', '69123','13055'))
df_arrgpy = df_com_arr.groupby(['CODREG', 'REG', 'CODDEP', 'COM', 'CODGEO'])['PTOT'].sum().reset_index()                    

df_com_pop = df_com_pop.append(df_arrgpy, ignore_index=True)
df_com_pop = df_com_pop.drop(df_com_pop[df_com_pop["COM"].str.contains('Arrondissement')].index)

/tmp/ipykernel_2518/477142194.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_com_arr['COM']=np.where(df_com_arr['CODDEP']=='75', 'Paris',
/tmp/ipykernel_2518/477142194.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_com_arr['CODGEO']=np.where(df_com_arr['CODDEP']=='75', '75056',
/tmp/ipykernel_2518/477142194.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_com_pop = df_com_pop.append(df_arrgpy, igno

Jointure fichiers df_com_shp, df_com_pop et df_ZAU (géométrie + population + aires d'attraction)

In [90]:
df_com = pd.merge(df_com_shp,df_com_pop[["CODGEO",'PTOT']],left_on='CODGEO', right_on='CODGEO')

In [91]:
df_com = pd.merge(df_com,df_ZAU[["CODGEO","CATEAAV2020",'AAV2020']],left_on='CODGEO', right_on='CODGEO')

Calcul de la population par aire d'attraction

In [92]:
#groupby des ZAU avec somme de la population
df_ZAUgpy = df_com.groupby(['AAV2020'])['PTOT'].sum().reset_index()

# Jointure du groupby avec les communes
df_com = pd.merge(df_com,df_ZAUgpy,left_on='AAV2020', right_on='AAV2020')

# On renomme les colonnes
df_com.rename(columns={"PTOT_x": "POPULATION", "PTOT_y": "POPPUZAU"},inplace=True)

# On supprime les colonnes inutiles
#df_com.drop(columns=["INSEE_CAN",'INSEE_ARR','ID','SIREN_EPCI'],inplace= True)
df_com.drop(columns=['aav2020'],inplace= True)

In [93]:
df_com = df_com.to_crs("EPSG:3857")

Exportation du résultat

In [95]:
df_com.to_file("./Data/com_zau.gpkg", layer='com_zau', driver="GPKG")

/home/manon/miniconda3/envs/stage/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## Partie 2 : intersection entre Corine Land Cover et com_zau. A faire sous Qgis